In [15]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Bidirectional
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import filter_data_by_year_month, prep_for_RNN

In [2]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/Historical Prices for BTCUSDT')


In [3]:
YEARS = [2019,2021,2022,2023,2024]

Data = filter_data_by_year_month(X, YEARS)

X1, y = prep_for_RNN(Data, "close", 3)



In [6]:
X1[0]

array([[3701.23    , 3703.72    , 3689.88    , 3697.31    ,  190.974095],
       [3697.91    , 3698.75    , 3690.56    , 3694.2     ,  131.742721],
       [3693.96    , 3705.      , 3693.      , 3702.39    ,  171.877714]])

In [5]:
y[0]

3700.31

In [11]:
print(np.any(np.isnan(X1)))  # Check features for NaNs
print(np.any(np.isnan(y)))

False
False


In [12]:
dataset = tf.data.Dataset.from_tensor_slices((X1, y))

In [16]:
# Calculate sizes for train and validation sets
dataset_size = len(X1)  # Total number of samples in your dataset
train_size = int(0.8 * dataset_size)

# Shuffle the dataset if needed
dataset = dataset.shuffle(buffer_size=dataset_size)

# Split the dataset
train_dataset = dataset.take(train_size)       # Take the first 80%
val_dataset = dataset.skip(train_size)         # Skip the first 80%, take the remaining 20%

# Batch both datasets if needed
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [31]:
for X_batch, y_batch in dataset.take(1):
    print("Batch input shape:", X_batch.shape)  # Expected: (batch_size, 3, 4)
    print("Batch target shape:", y_batch.shape)

Batch input shape: (3, 5)
Batch target shape: ()


2025-01-22 13:57:45.390544: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [32]:
tf.random.set_seed(1234) # for consistent results
model = Sequential([
    GRU(64, input_shape=(3, 4), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Single output for regression
])

In [23]:
model.summary()

Model: "simplified_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                     │ (None, 3, 64)          │        13,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 3, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_Layer (Dense)             │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,937 (93.50 KB)

 Trainable params: 23,937 (93.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = tf.keras.models.load_model("../models/")

In [34]:
model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['mae']
)


reduce_lr = ReduceLROnPlateau(
    monitor='val_mae',  # Choose the metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=3,          # Number of epochs with no improvement before reducing
    min_lr=1e-6          # Minimum learning rate
)

checkpoint = ModelCheckpoint(
    '../models/DNN_model_1.keras',             # File to save the best model
    monitor='val_mae',       # Metric to monitor for improvement
    mode='min',                   # Mode 'min' for mae (since lower is better)
    save_best_only=True,          # Save only when there is an improvement
    verbose=1,                    # Print message when saving
                    
)


history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=50,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/50
4370/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 490412512.0000 - mae: 18177.3945
Epoch 1: val_mae improved from inf to 18078.35742, saving model to ../models/DNN_model_1.keras
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 490415680.0000 - mae: 18177.4648 - val_loss: 484447072.0000 - val_mae: 18078.3574 - learning_rate: 0.0100
Epoch 2/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 490397568.0000 - mae: 18183.2832
Epoch 2: val_mae did not improve from 18078.35742
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 490397824.0000 - mae: 18183.2871 - val_loss: 487486112.0000 - val_mae: 18096.6621 - learning_rate: 0.0100
Epoch 3/50
4370/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 495142112.0000 - mae: 18269.4414
Epoch 3: val_mae did not improve from 18078.35742
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 495136192.0000 - mae: 18269.3359 - val_loss: 484420128.0000 - val_mae: 18117.6504 - learning_rate: 0.0100
Epoch 4/50
4378/4378 ━━━━━━━━━━━━━━━━━━━━ 0s 3